# Data load 

# Environment

---

In [82]:
#!pip install pandas sqlalchemy psycopg2 # Postgres DB library

import pandas as pd
from sqlalchemy import create_engine
import json
from sqlalchemy import text
import re

In [106]:
# Cargar el DataFrame
df = pd.read_csv("../data/Airbnb_Open_Data.csv", low_memory=False, encoding='ISO-8859-1')

# Función para identificar valores no ASCII
def reemplazar_no_ascii(valor):
    if isinstance(valor, str) and not valor.isascii():
        return "999999"
    return valor

# Función para limpiar caracteres especiales en texto
def limpiar_texto(texto):
    if isinstance(texto, str) and not re.match(r'^[A-Za-z0-9 .,!?]+$', texto):
        return "999999"
    return texto

# Función para validar datos numéricos
def validar_numericos(valor):
    try:
        num = float(valor)
        if num < 0 or num > 99999999:  # Puedes ajustar este umbral según el caso
            return 999999
    except ValueError:
        return 999999  # Si no es un número, lo marcamos como inconsistente
    return num

# Aplicar validaciones en todas las columnas según su tipo
for col in df.columns:
    if df[col].dtype == "object":  # Columnas de texto
        df[col] = df[col].map(reemplazar_no_ascii)
        df[col] = df[col].map(limpiar_texto)
    else:  # Columnas numéricas
        df[col] = df[col].map(validar_numericos)

# Reemplazar NaN e infinitos con 999999
df = df.fillna(999999)
df.replace([float('inf'), float('-inf')], 999999, inplace=True)

# Mostrar primeras filas
print(df.head())

In [107]:
with open("../credentials.json") as f:
    creds = json.load(f)
    
DB_USER = creds["user"]
DB_PASSWORD = creds["password"]
DB_HOST = creds["host"]
DB_PORT = creds["port"]
DB_NAME = creds["database"]

connection_string = f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}?client_encoding=latin1'

In [108]:
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}?client_encoding=utf8")
df.to_sql("airbnb", engine, if_exists='append', index=False)

In [97]:
query = "SELECT * FROM airbnb;"

df = pd.read_sql(query, engine)

print(df.head())

In [120]:
# Importar bibliotecas necesarias
import pandas as pd
from sqlalchemy import create_engine, text
import json

# Cargar las credenciales desde el archivo JSON
with open("../credentials.json") as f:
    creds = json.load(f)
    
DB_USER = creds["user"]
DB_PASSWORD = creds["password"]
DB_HOST = creds["host"]
DB_PORT = creds["port"]
DB_NAME = "airbnb_csv_db"  # Nombre de la nueva base de datos

In [121]:
# Leer el archivo CSV
df = pd.read_csv("../data/Airbnb_Open_Data.csv", low_memory=False, encoding='ISO-8859-1')
print("Datos del CSV cargados correctamente.")
print(df.head())  # Mostrar las primeras filas del DataFrame

In [116]:
# Crear la cadena de conexión a la base de datos predeterminada (postgres)
connection_string = f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/postgres'
engine = create_engine(connection_string)

# Verificar si la base de datos ya existe
with engine.connect() as connection:
    connection.execution_options(isolation_level="AUTOCOMMIT")
    
    # Consultar la lista de bases de datos
    result = connection.execute(text("SELECT datname FROM pg_database;"))
    databases = [row[0] for row in result]

    if DB_NAME in databases:
        print(f"La base de datos '{DB_NAME}' ya existe.")
    else:
        # Crear la nueva base de datos si no existe
        connection.execute(text(f"CREATE DATABASE {DB_NAME};"))
        print(f"Base de datos '{DB_NAME}' creada.")

In [117]:
# Crear la cadena de conexión para la nueva base de datos
new_connection_string = f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
new_engine = create_engine(new_connection_string)

# Crear la tabla en la nueva base de datos y cargar los datos
table_name = "airbnb_data"  # Nombre de la nueva tabla
df.to_sql(table_name, new_engine, if_exists='replace', index=False)

print(f"Tabla '{table_name}' creada y datos cargados en la base de datos '{DB_NAME}'.")

In [119]:
# Verificar los datos en la nueva tabla
with new_engine.connect() as connection:
    result = connection.execute(text(f"SELECT * FROM {table_name} LIMIT 5;"))
    for row in result:
        print(row)